In [1]:
import pandas as pd

In [2]:
dataset_name = 'mapbiomas_cobertura_col8'
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

### Bronze

In [3]:
(pd.read_excel('https://storage.googleapis.com/mapbiomas-public/initiatives/brasil/collection_8/downloads/statistics/tabela_geral_mapbiomas_col8_biomas_municipios.xlsx',
               sheet_name = 1)
 .to_excel(f'{raiz}/bronze_{dataset_name}.xlsx',
           index=False)
)

### Fase Prata

|---------------------------------------------------------------------|
|                              COLEÇÃO 8 - CLASSES                              |
|-------------------------------------|---------------------------------|-------|
| Categoria                           | Subcategoria                    | ID    |
|-------------------------------------|---------------------------------|-------|
| 3.1. Pastagem                       |                                 | 15    |
| 3.2. Agricultura                    |                                 | 18    |
|                                     | 3.2.1. Lavoura Temporária      | 19    |
|                                     |     3.2.1.1. Soja             | 39    |
|                                     |     3.2.1.2. Cana             | 20    |
|                                     |     3.2.1.3. Arroz (beta)    | 40    |
|                                     |     3.2.1.4. Algodão (beta)  | 62    |
|                                     |     3.2.1.5. Outras Lavouras Temporárias | 41 |
|                                     | 3.2.2. Lavoura Perene          | 36    |
|                                     |     3.2.2.1. Café             | 46    |
|                                     |     3.2.2.2. Citrus           | 47    |
|                                     |     3.2.2.3. Dendê           | 35    |
|                                     |     3.2.1.3. Outras Lavouras Perenes | 48 |
| 3.4. Mosaico de Usos                |                                 | 21    |


Descrição dos códigos: https://brasil.mapbiomas.org/wp-content/uploads/sites/4/2023/08/Legenda-Colecao-8-LEGEND-CODE.pdf

In [4]:
anos = (pd.read_excel(f'{raiz}/bronze_{dataset_name}.xlsx',
               usecols=lambda x: isinstance(x, int))
        .columns
        .drop(range(1985, 2000))
        .tolist()
        )

In [5]:
df = (pd.read_excel(f'{raiz}/bronze_{dataset_name}.xlsx')
    .drop(range(1985, 2000), axis = 'columns')
    .query('class_id in (15, 18, 19, 39, 20, 40, 62, 41, 36, 46, 47, 35, 48, 21)')
    .assign(**{
        'uf': lambda df: (df['geocode']// 100000)
        }
            )
    .melt(
        id_vars=['uf', 'geocode', 'class_id'],
        value_vars=anos,
        var_name='ano',
        value_name='area_ha'
        )
    .dropna(subset='area_ha')
    .convert_dtypes()
    .rename(columns=
            {
                'geocode':'municipio',
                'class_id':'classe_solo'
             })
    .set_index(['uf', 'municipio', 'ano', 'classe_solo'])
    )

In [6]:
df.to_parquet(f'{raiz}/prata_{dataset_name}_areaclass_agropecuaria.parquet')

In [7]:
(df.reset_index()
 .assign(**{
          'fonte': 'Deposição Atmosférica'
          })
 .groupby(['uf', 'municipio', 'ano', 'fonte'])
 .agg({'area_ha':sum})
 .to_parquet(f'{raiz}/prata_{dataset_name}_areatotal_agropecuaria.parquet')
 )

### Fase Ouro

Área Destinada a Agropecuária (ha)
Elementos (kg/ha ano)

Cálculo: Cobertura Solo Municipal * (Parâmetro_X/1000)

UF e Brasil = mil t

NPK oriundo de deposição atmosférica incorporado ao solo (t)

In [8]:
parametro_N = 5.5
parametro_P = 0.3
parametro_K = 3.1

In [13]:
df = (pd.read_parquet(f'{raiz}/prata_{dataset_name}_areatotal_agropecuaria.parquet')
      .assign(**{
          'N': lambda df: df['area_ha']*(parametro_N/1000),
          'P': lambda df: df['area_ha']*(parametro_P/1000),
          'K': lambda df: df['area_ha']*(parametro_K/1000)
          })
      .drop('area_ha', axis = 1)
      .reset_index()
      .convert_dtypes()
      .melt(id_vars=['uf', 'ano', 'fonte'],
            value_vars=['N', 'P', 'K'],
            var_name='elemento',
            value_name='valor')
     .groupby(['uf', 'ano', 'fonte', 'elemento'])
     .agg({'valor':sum})
     .reset_index(level='elemento')
     .sort_values(by=['elemento', 'uf'],
                   key=lambda x: x.map({'N': 0, 'P': 1, 'K': 2}))
)

df.to_csv(f'{raiz}/ouro_npk_deposicao_atmosferica.csv')

/tmp/ipykernel_51983/1843221.py:1: FutureWarning: The argument 'use_nullable_dtypes' is deprecated and will be removed in a future version.Use dtype_backend='numpy_nullable' instead of use_nullable_dtype=True.
  df = (pd.read_parquet(f'{raiz}/prata_mapbiomas_areatotal_agropecuaria.parquet',


In [14]:
df

elemento         valor
uf ano  fonte                                       
11 2000 Deposição Atmosférica        K  15943.223744
        Deposição Atmosférica        N  28286.364707
        Deposição Atmosférica        P    1542.89262
   2001 Deposição Atmosférica        K  16776.124492
        Deposição Atmosférica        N   29764.09184
...                                ...           ...
53 2021 Deposição Atmosférica        N    1488.10944
        Deposição Atmosférica        P     81.169606
   2022 Deposição Atmosférica        K     826.18947
        Deposição Atmosférica        N   1465.820028
        Deposição Atmosférica        P      79.95382

[1863 rows x 2 columns]